In [165]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import lit
import locale
import sys
import os
import re

In [166]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [167]:
spark

# Leyendo lista de Colegios y Estudiantes

In [204]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [205]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [206]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [207]:
print("Archivos de estudiantes: ", estuDic['20141'],estuDic['20142'])
print("Archivos de colegios: ",coleDic['20141'],coleDic['20142'])

Archivos de estudiantes:  SB11_20141.txt SB11_20142.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-20141.txt SB11-CLASIFI-PLANTELES-20142.txt


In [208]:
df1 = spark.read.load(base2 + '/' + estuDic['20141'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [209]:
df1.columns

['ESTU_TIPODOCUMENTO',
 'ESTU_NACIONALIDAD',
 'ESTU_GENERO',
 'ESTU_FECHANACIMIENTO',
 'PERIODO',
 'ESTU_CONSECUTIVO',
 'ESTU_ESTUDIANTE',
 'ESTU_PAIS_RESIDE',
 'ESTU_DEPTO_RESIDE',
 'ESTU_COD_RESIDE_DEPTO',
 'ESTU_MCPIO_RESIDE',
 'ESTU_COD_RESIDE_MCPIO',
 'ESTU_AREARESIDE',
 'ESTU_VALORPENSIONCOLEGIO',
 'ESTU_VECESPRESENTOEXAMEN',
 'FAMI_EDUCACIONPADRE',
 'FAMI_EDUCACIONMADRE',
 'FAMI_OCUPACIONPADRE',
 'FAMI_OCUPACIONMADRE',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_NIVELSISBEN',
 'FAMI_PERSONASHOGAR',
 'FAMI_CUARTOSHOGAR',
 'FAMI_PISOSHOGAR',
 'FAMI_TIENEINTERNET',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEMICROONDAS',
 'FAMI_TIENEHORNO',
 'FAMI_TIENETELEVISOR',
 'FAMI_TIENEAUTOMOVIL',
 'FAMI_TIENEDVD',
 'FAMI_TIENE_NEVERA',
 'FAMI_TIENE_CELULAR',
 'FAMI_TELEFONO',
 'FAMI_INGRESOFMILIARMENSUAL',
 'ESTU_TRABAJAACTUALMENTE',
 'ESTU_RECIBESALARIO',
 'ESTU_ANTECEDENTES',
 'ESTU_ANOSPREESCOLAR',
 'ESTU_ANOMATRICULAPRIMERO',
 'ESTU_ANOTERMINOQUINTO',
 'ESTU_ANOMATRICULASEXTO',
 'ES

In [210]:
columnasPuntos20141 = [
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'PUNT_MATEMATICAS',
 'PUNT_PROFUNDIZA_MATEMATICA',
 'PUNT_CIENCIAS_SOCIALES',
 'PUNT_PROFUNDIZA_CSOCIALES',
 'PUNT_INGLES',
 'PUNT_BIOLOGIA',
 'PUNT_PROFUNDIZA_BIOLOGIA',
 'PUNT_FILOSOFIA',
 'PUNT_INTERDISC_MEDIOAMBIENTE',
 'PUNT_FISICA',
 'PUNT_QUIMICA',
 'PUNT_LENGUAJE',
 'PUNT_PROFUNDIZA_LENGUAJE',
 'PUNT_INTERDISC_VIOLENCIAYSOC']

In [211]:
df1 = df1.select(columnasPuntos20141)

In [212]:
# SE leen los dos archivos y se crean dos dataframes
df2 = spark.read.load(base2 + '/' + estuDic['20142'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [213]:
columnasPuntos20142 = [
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'PUNT_LECTURA_CRITICA',
 'PUNT_MATEMATICAS',
 'PUNT_C_NATURALES',
 'PUNT_SOCIALES_CIUDADANAS',
 'PUNT_RAZONA_CUANTITATIVO',
 'PUNT_COMP_CIUDADANA',
 'PUNT_INGLES'
]

In [214]:
df2 = df2.select(columnasPuntos20142)

In [215]:
#dft = df1.columns.remove('cole_cod_dane_institucion')
cols = df1.columns
cols.remove('COLE_COD_DANE_ESTABLECIMIENTO')
cols.remove('COLE_NOMBRE_ESTABLECIMIENTO')
cols

['PUNT_MATEMATICAS',
 'PUNT_PROFUNDIZA_MATEMATICA',
 'PUNT_CIENCIAS_SOCIALES',
 'PUNT_PROFUNDIZA_CSOCIALES',
 'PUNT_INGLES',
 'PUNT_BIOLOGIA',
 'PUNT_PROFUNDIZA_BIOLOGIA',
 'PUNT_FILOSOFIA',
 'PUNT_INTERDISC_MEDIOAMBIENTE',
 'PUNT_FISICA',
 'PUNT_QUIMICA',
 'PUNT_LENGUAJE',
 'PUNT_PROFUNDIZA_LENGUAJE',
 'PUNT_INTERDISC_VIOLENCIAYSOC']

In [216]:
# SE unen los dos dataframes
# df = unionAll(df1,df2)
for cp in cols:
    df1 = df1.withColumn(cp, regexp_replace(cp, ',', '.'))
    df1 = df1.withColumn(cp, df1[cp].cast("float"))
    df1 = df1.na.fill({cp:0})
df1 = df1.na.fill({'COLE_COD_DANE_ESTABLECIMIENTO':'--'})
df1 = df1.na.fill({'COLE_NOMBRE_ESTABLECIMIENTO':'--'})

In [217]:
# SE evalua sí tiene valores nullos
print("Nùmero de filas con cole_cod_dane_institucion nulos: ", df1.filter("COLE_COD_DANE_ESTABLECIMIENTO = '--'").count())
print("Nùmero de filas con cole_nombre_sede nulos: ", df1.filter("COLE_NOMBRE_ESTABLECIMIENTO = '--'").count())

Nùmero de filas con cole_cod_dane_institucion nulos:  0
Nùmero de filas con cole_nombre_sede nulos:  0


In [218]:
df1 = df1.withColumn('COLE_COD_DANE_ESTABLECIMIENTO', df1['COLE_COD_DANE_ESTABLECIMIENTO'].cast("string"))

In [219]:
cols2 = df2.columns
cols2.remove('COLE_COD_DANE_ESTABLECIMIENTO')
cols2.remove('COLE_NOMBRE_ESTABLECIMIENTO')

In [220]:
for cp in cols2:
    df2 = df2.withColumn(cp, regexp_replace(cp, ',', '.'))
    df2 = df2.withColumn(cp, df2[cp].cast("float"))
    df2 = df2.na.fill({cp:0})
df2 = df2.withColumn('COLE_COD_DANE_ESTABLECIMIENTO', df2['COLE_COD_DANE_ESTABLECIMIENTO'].cast("string"))
df2 = df2.na.fill({'COLE_COD_DANE_ESTABLECIMIENTO':'--'})
df2 = df2.na.fill({'COLE_NOMBRE_ESTABLECIMIENTO':'--'})

In [221]:
# SE evalua sí tiene valores nullos
print("Nùmero de filas con cole_cod_dane_institucion nulos: ", df2.filter("COLE_COD_DANE_ESTABLECIMIENTO = '--'").count())
print("Nùmero de filas con cole_nombre_sede nulos: ", df2.filter("COLE_NOMBRE_ESTABLECIMIENTO = '--'").count())

Nùmero de filas con cole_cod_dane_institucion nulos:  107
Nùmero de filas con cole_nombre_sede nulos:  0


In [222]:
print(df1.printSchema())

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: string (nullable = true)
 |-- COLE_NOMBRE_ESTABLECIMIENTO: string (nullable = false)
 |-- PUNT_MATEMATICAS: float (nullable = false)
 |-- PUNT_PROFUNDIZA_MATEMATICA: float (nullable = false)
 |-- PUNT_CIENCIAS_SOCIALES: float (nullable = false)
 |-- PUNT_PROFUNDIZA_CSOCIALES: float (nullable = false)
 |-- PUNT_INGLES: float (nullable = false)
 |-- PUNT_BIOLOGIA: float (nullable = false)
 |-- PUNT_PROFUNDIZA_BIOLOGIA: float (nullable = false)
 |-- PUNT_FILOSOFIA: float (nullable = false)
 |-- PUNT_INTERDISC_MEDIOAMBIENTE: float (nullable = false)
 |-- PUNT_FISICA: float (nullable = false)
 |-- PUNT_QUIMICA: float (nullable = false)
 |-- PUNT_LENGUAJE: float (nullable = false)
 |-- PUNT_PROFUNDIZA_LENGUAJE: float (nullable = false)
 |-- PUNT_INTERDISC_VIOLENCIAYSOC: float (nullable = false)

None


In [223]:
print(df2.printSchema())

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: string (nullable = false)
 |-- COLE_NOMBRE_ESTABLECIMIENTO: string (nullable = false)
 |-- PUNT_LECTURA_CRITICA: float (nullable = false)
 |-- PUNT_MATEMATICAS: float (nullable = false)
 |-- PUNT_C_NATURALES: float (nullable = false)
 |-- PUNT_SOCIALES_CIUDADANAS: float (nullable = false)
 |-- PUNT_RAZONA_CUANTITATIVO: float (nullable = false)
 |-- PUNT_COMP_CIUDADANA: float (nullable = false)
 |-- PUNT_INGLES: float (nullable = false)

None


In [224]:
# Contando el numero de estudiantes por instituciòn y mostrandola de forma descendente
dsCountEst1 = df1.groupBy('COLE_COD_DANE_ESTABLECIMIENTO').count().orderBy('count',ascending=
False)
dsCountEst2 = df2.groupBy('COLE_COD_DANE_ESTABLECIMIENTO').count().orderBy('count',ascending=
False)

In [225]:
dsCountEst1 = dsCountEst1.withColumnRenamed("count","num_est")
dsCountEst1.show(10,False)

dsCountEst2 = dsCountEst2.withColumnRenamed("count","num_est")
dsCountEst2.show(10,False)

+-----------------------------+-------+
|COLE_COD_DANE_ESTABLECIMIENTO|num_est|
+-----------------------------+-------+
|311001105863                 |465    |
|305001024544                 |326    |
|305001007861                 |267    |
|476248000478                 |258    |
|376001028219                 |204    |
|376001000721                 |176    |
|376001003461                 |168    |
|311001106410                 |159    |
|376001007670                 |150    |
|376001026925                 |142    |
+-----------------------------+-------+
only showing top 10 rows

+-----------------------------+-------+
|COLE_COD_DANE_ESTABLECIMIENTO|num_est|
+-----------------------------+-------+
|311001105391                 |1133   |
|105001000108                 |1001   |
|311001105863                 |957    |
|105001013340                 |794    |
|105615000236                 |671    |
|305001007861                 |663    |
|176001005813                 |648    |
|305001003963 

In [226]:
rowmax1 = dsCountEst1.agg({"num_est": "max"}).collect()[0]
rowmin1 = dsCountEst1.agg({"num_est": "min"}).collect()[0]

num_est_max1 = rowmax1["max(num_est)"]
num_est_min1 = rowmin1["min(num_est)"]

print("Numero màximo de estudiantes: ",num_est_max1)
print("Numero mìnimo de estudiantes: ",num_est_min1)

Numero màximo de estudiantes:  465
Numero mìnimo de estudiantes:  1


In [227]:
rowmax2 = dsCountEst2.agg({"num_est": "max"}).collect()[0]
rowmin2 = dsCountEst2.agg({"num_est": "min"}).collect()[0]

num_est_max2 = rowmax2["max(num_est)"]
num_est_min2 = rowmin2["min(num_est)"]

print("Numero màximo de estudiantes: ",num_est_max2)
print("Numero mìnimo de estudiantes: ",num_est_min2)

Numero màximo de estudiantes:  1133
Numero mìnimo de estudiantes:  1


In [228]:
#Se crea una columna con el total de los puntajes 
df1= df1.withColumn("Suma",df1['PUNT_MATEMATICAS']+df1['PUNT_PROFUNDIZA_MATEMATICA']+df1['PUNT_CIENCIAS_SOCIALES']+
                     df1['PUNT_PROFUNDIZA_CSOCIALES']+df1['PUNT_INGLES']+df1['PUNT_BIOLOGIA']+df1['PUNT_PROFUNDIZA_BIOLOGIA']+
                     df1['PUNT_FILOSOFIA']+df1['PUNT_INTERDISC_MEDIOAMBIENTE']+df1['PUNT_FISICA']+df1['PUNT_QUIMICA']+
                     df1['PUNT_LENGUAJE']+df1['PUNT_PROFUNDIZA_LENGUAJE']+df1['PUNT_INTERDISC_VIOLENCIAYSOC'])

df2= df2.withColumn("Suma",df2['PUNT_LECTURA_CRITICA']+df2['PUNT_MATEMATICAS']+df2['PUNT_C_NATURALES']
                    +df2['PUNT_SOCIALES_CIUDADANAS']+df2['PUNT_RAZONA_CUANTITATIVO']+df2['PUNT_COMP_CIUDADANA']
                    +df2['PUNT_INGLES'])

## Resultados finales por instituciòn 1er semestre

In [229]:
# Còdigo para obtener la media de los resultados de las evaluaciones de todos los estudiantes por instituciòn 
# y posiciòn en orden descendente.
from pyspark.sql.functions import *
from pyspark.sql.window import Window

#df = df.groupBy('cole_cod_dane_institucion','cole_nombre_sede').mean()
df1 = df1.groupBy('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO').mean()

rowprommax1 = df1.agg({"avg(Suma)": "max"}).collect()[0]
rowprommin1 = df1.agg({"avg(Suma)": "min"}).collect()[0]

prom_est_max1 = rowprommax1["max(avg(Suma))"]
prom_est_min1 = rowprommin1["min(avg(Suma))"]

print("Màxima calificaciòn en colegios: ", prom_est_max1)
print("Mìxima calificaciòn en colegios: ", prom_est_min1)

Màxima calificaciòn en colegios:  594.719298245614
Mìxima calificaciòn en colegios:  271.0


In [230]:
dfjoin = df1.join(dsCountEst1,"COLE_COD_DANE_ESTABLECIMIENTO")
dfjoin= dfjoin.withColumn("Indice_Prom",round(dfjoin['avg(Suma)']/prom_est_max1,3))
dfjoin= dfjoin.withColumn("Indice_Num",round(dfjoin['num_est']/num_est_max1,3))
dfjoin= dfjoin.withColumn("Indice_Total",round(dfjoin['Indice_Prom']*dfjoin['Indice_Num'],3))

In [231]:
dfjoin = df1.join(dsCountEst1,"COLE_COD_DANE_ESTABLECIMIENTO")
dfjoin= dfjoin.withColumn("Indice_Prom",round(dfjoin['avg(Suma)']/prom_est_max1,3))
dfjoin= dfjoin.withColumn("Indice_Num",round(dfjoin['num_est']/num_est_max1,3))
dfjoin= dfjoin.withColumn("Indice_Total",round(dfjoin['Indice_Prom']*dfjoin['Indice_Num'],3))

In [232]:
dfjoin = dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','avg(Suma)','num_est','Indice_Prom','Indice_Num','Indice_Total')
dfjoin = dfjoin.withColumn("rankProm", dense_rank().over(Window.orderBy(desc("Indice_Prom"))))
dfjoin = dfjoin.withColumn("rankNum", dense_rank().over(Window.orderBy(desc("Indice_Num"))))
dfjoin = dfjoin.withColumn("rankTotal", dense_rank().over(Window.orderBy(desc("Indice_Total"))))
dfjoin = dfjoin.withColumn("Media",round(dfjoin['avg(Suma)'],2))

In [233]:
# Se ordena por Indice_Prom
dfjoin = dfjoin.orderBy('rankProm',ascending= True)
dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','Media','Indice_Prom','num_est','rankProm','rankNum','rankTotal').show(10)

+---------------------------+------+-----------+-------+--------+-------+---------+
|COLE_NOMBRE_ESTABLECIMIENTO| Media|Indice_Prom|num_est|rankProm|rankNum|rankTotal|
+---------------------------+------+-----------+-------+--------+-------+---------+
|            COL LOS NOGALES|594.72|        1.0|     57|       1|     62|       50|
|       COL LA QUINTA DEL...|586.77|      0.987|     30|       2|     89|      100|
|       COL SAN MATEO APO...|576.21|      0.969|     61|       3|     58|       47|
|       COL  MONTESSORI B...|570.43|      0.959|     35|       4|     84|       93|
|       COL SAN CARLOS   ...|569.99|      0.958|     92|       5|     30|       22|
|       GIMN VERMONT     ...|569.82|      0.958|    103|       5|     23|       16|
|        ASPAEN LICEO TACURÍ|567.62|      0.954|     13|       6|    106|      134|
|       ASPAEN GIMNASIO C...|564.48|      0.949|     33|       7|     86|       97|
|       COL SAN JORGE DE ...|563.18|      0.947|     88|       8|     32|   

In [234]:
dfjoin = dfjoin.orderBy('rankNum',ascending= True)
dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','Media','Indice_Num','num_est','rankProm','rankNum','rankTotal').show(10)

+---------------------------+------+----------+-------+--------+-------+---------+
|COLE_NOMBRE_ESTABLECIMIENTO| Media|Indice_Num|num_est|rankProm|rankNum|rankTotal|
+---------------------------+------+----------+-------+--------+-------+---------+
|       CENT JOHANN KEPLE...|359.25|       1.0|    465|     244|      1|        1|
|       COL NUEVO CERVANT...|326.86|     0.701|    326|     298|      2|        2|
|            GIMN GUAYACANES|348.07|     0.574|    267|     263|      3|        3|
|       INST TEC CIAL E I...|356.78|     0.555|    258|     248|      4|        4|
|       COLEGIO COMFANDI ...|433.47|     0.439|    204|     138|      5|        5|
|       COLEGIO FRANCISCA...|426.08|     0.378|    176|     145|      6|        7|
|       COLEGIO SAN ANTON...|471.26|     0.361|    168|      95|      7|        6|
|       INST POLITECNICO ...|337.51|     0.342|    159|     280|      8|       20|
|       COLEGIO LEON DE G...|437.57|     0.323|    150|     131|      9|        9|
|   

In [235]:
# Se ordena por Indice_Total
dfjoin = dfjoin.orderBy('rankTotal',ascending= True)
dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','Media','Indice_Total','num_est','rankProm','rankNum','rankTotal').show(10)

+---------------------------+------+------------+-------+--------+-------+---------+
|COLE_NOMBRE_ESTABLECIMIENTO| Media|Indice_Total|num_est|rankProm|rankNum|rankTotal|
+---------------------------+------+------------+-------+--------+-------+---------+
|       CENT JOHANN KEPLE...|359.25|       0.604|    465|     244|      1|        1|
|       COL NUEVO CERVANT...|326.86|       0.386|    326|     298|      2|        2|
|            GIMN GUAYACANES|348.07|       0.336|    267|     263|      3|        3|
|       INST TEC CIAL E I...|356.78|       0.333|    258|     248|      4|        4|
|       COLEGIO COMFANDI ...|433.47|        0.32|    204|     138|      5|        5|
|       COLEGIO SAN ANTON...|471.26|       0.286|    168|      95|      7|        6|
|       COLEGIO FRANCISCA...|426.08|       0.271|    176|     145|      6|        7|
|       COL SAN BARTOLOME...| 523.7|       0.239|    126|      45|     15|        8|
|       COLEGIO LEON DE G...|437.57|       0.238|    150|     131

## Resultados finales por instituciòn 2do semestre

In [236]:
# Còdigo para obtener la media de los resultados de las evaluaciones de todos los estudiantes por instituciòn 
# y posiciòn en orden descendente.
from pyspark.sql.functions import *
from pyspark.sql.window import Window

#df = df.groupBy('cole_cod_dane_institucion','cole_nombre_sede').mean()
df2 = df2.groupBy('COLE_NOMBRE_ESTABLECIMIENTO','COLE_COD_DANE_ESTABLECIMIENTO').mean()

rowprommax2 = df2.agg({"avg(Suma)": "max"}).collect()[0]
rowprommin2 = df2.agg({"avg(Suma)": "min"}).collect()[0]

prom_est_max2 = rowprommax1["max(avg(Suma))"]
prom_est_min2 = rowprommin1["min(avg(Suma))"]

print("Màxima calificaciòn en colegios: ", prom_est_max2)
print("Mìxima calificaciòn en colegios: ", prom_est_min2)

Màxima calificaciòn en colegios:  594.719298245614
Mìxima calificaciòn en colegios:  271.0


In [238]:
dfjoin = df2.join(dsCountEst2,"COLE_COD_DANE_ESTABLECIMIENTO")
dfjoin= dfjoin.withColumn("Indice_Prom",round(dfjoin['avg(Suma)']/prom_est_max2,3))
dfjoin= dfjoin.withColumn("Indice_Num",round(dfjoin['num_est']/num_est_max2,3))
dfjoin= dfjoin.withColumn("Indice_Total",round(dfjoin['Indice_Prom']*dfjoin['Indice_Num'],3))

In [239]:
dfjoin = dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','avg(Suma)','num_est','Indice_Prom','Indice_Num','Indice_Total')
dfjoin = dfjoin.withColumn("rankProm", dense_rank().over(Window.orderBy(desc("Indice_Prom"))))
dfjoin = dfjoin.withColumn("rankNum", dense_rank().over(Window.orderBy(desc("Indice_Num"))))
dfjoin = dfjoin.withColumn("rankTotal", dense_rank().over(Window.orderBy(desc("Indice_Total"))))
dfjoin = dfjoin.withColumn("Media",round(dfjoin['avg(Suma)'],2))

In [240]:
# Se ordena por Indice_Prom
dfjoin = dfjoin.orderBy('rankProm',ascending= True)
dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','Media','Indice_Prom','num_est','rankProm','rankNum','rankTotal').show(10)

+---------------------------+------+-----------+-------+--------+-------+---------+
|COLE_NOMBRE_ESTABLECIMIENTO| Media|Indice_Prom|num_est|rankProm|rankNum|rankTotal|
+---------------------------+------+-----------+-------+--------+-------+---------+
|       COLOMBO BRITANICO...|541.58|      0.911|     72|       1|    252|      155|
|       CAMBRIDGE SCHOOL ...|538.57|      0.906|     14|       2|    304|      202|
|                LIC NAVARRA|534.09|      0.898|     45|       3|    276|      177|
|        INST ALBERTO MERANI|530.41|      0.892|     32|       4|    288|      188|
|       COL NUEVO COLOMBO...|527.86|      0.888|     65|       5|    259|      162|
|            LIC CAMPO DAVID|526.35|      0.885|     17|       6|    301|      200|
|        COL ANGLO AMERICANO|524.37|      0.882|    166|       7|    169|       83|
|                 COL REFOUS|523.81|      0.881|     72|       8|    252|      157|
|        COLEGIO CORAZONISTA|519.91|      0.874|    149|       9|    184|   

In [241]:
dfjoin = dfjoin.orderBy('rankNum',ascending= True)
dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','Media','Indice_Num','num_est','rankProm','rankNum','rankTotal').show(10)

+---------------------------+------+----------+-------+--------+-------+---------+
|COLE_NOMBRE_ESTABLECIMIENTO| Media|Indice_Num|num_est|rankProm|rankNum|rankTotal|
+---------------------------+------+----------+-------+--------+-------+---------+
|       COL TECNISISTEMAS...|337.34|       1.0|   1133|     278|      1|        1|
|             INST EDUC CEFA|376.28|     0.883|   1001|     212|      2|        2|
|         CENT JOHANN KEPLER|320.51|     0.845|    957|     306|      3|        4|
|       INST EDUC INEM JO...|388.74|     0.701|    794|     191|      4|        3|
|       I. E. ESCUELA NOR...|316.58|     0.592|    671|     313|      5|        8|
|            GIMN GUAYACANES|316.58|     0.585|    663|     313|      6|        9|
|       INSTITUCION EDUCA...|367.61|     0.572|    648|     227|      7|        6|
|       LIC SALAZAR Y HER...|399.25|     0.537|    608|     174|      8|        5|
|       I. E. LICEO CAUCASIA|331.38|     0.505|    572|     288|      9|       13|
|   

In [242]:
# Se ordena por Indice_Total
dfjoin = dfjoin.orderBy('rankTotal',ascending= True)
dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','Media','Indice_Total','num_est','rankProm','rankNum','rankTotal').show(10)

+---------------------------+------+------------+-------+--------+-------+---------+
|COLE_NOMBRE_ESTABLECIMIENTO| Media|Indice_Total|num_est|rankProm|rankNum|rankTotal|
+---------------------------+------+------------+-------+--------+-------+---------+
|       COL TECNISISTEMAS...|337.34|       0.567|   1133|     278|      1|        1|
|             INST EDUC CEFA|376.28|       0.559|   1001|     212|      2|        2|
|       INST EDUC INEM JO...|388.74|       0.458|    794|     191|      4|        3|
|         CENT JOHANN KEPLER|320.51|       0.455|    957|     306|      3|        4|
|       LIC SALAZAR Y HER...|399.25|        0.36|    608|     174|      8|        5|
|       INSTITUCION EDUCA...|367.61|       0.353|    648|     227|      7|        6|
|       COLEGIO DE LA UNI...|424.35|       0.342|    543|     131|     11|        7|
|       I. E. ESCUELA NOR...|316.58|       0.315|    671|     313|      5|        8|
|            GIMN GUAYACANES|316.58|       0.311|    663|     313

## Colegios primer semestre

In [63]:
# SE crea Dataframe de colegios primer semestre
dfc = spark.read.load(base + '/' + coleDic['20141'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='\t')
dfc = dfc.withColumn('COLE_COD_DANE', dfc['COLE_COD_DANE'].cast("string"))

In [64]:
print(dfc.printSchema())

root
 |-- PERIODO: integer (nullable = true)
 |-- COLE_COD_DANE: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: integer (nullable = true)
 |-- COLE_MPIO_MUNICIPIO: string (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_INST_JORNADA: string (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_QUIMICA: integer (nullable = true)
 |-- COLE_FISICA: integer (nullable = true)
 |-- COLE_BIOLOGIA: integer (nullable = true)
 |-- COLE_FILOSOFIA: integer (nullable = true)
 |-- COLE_MATEMATICAS: integer (nullable = true)
 |-- COLE_LENGUAJE: integer (nullable = true)
 |-- COLE_CIENCIAS_SOCIALES: integer (nullable = true)
 |-- COLE_INGLES: integer (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- COLE_ESTUDIANTES_PRESENTES: integer (nullable = true)

None


In [65]:
colsColegio = dfc.columns

In [66]:
dfc.select('COLE_COD_DANE','COLE_INST_NOMBRE','COLE_ESTUDIANTES_PRESENTES').show(10)

+-------------+--------------------+--------------------------+
|COLE_COD_DANE|    COLE_INST_NOMBRE|COLE_ESTUDIANTES_PRESENTES|
+-------------+--------------------+--------------------------+
|        86769|COLEGIO SAN LUIS ...|                         7|
|        36889|COLEGIO LOS ANGEL...|                        52|
|       150581|  COLEGIO SAN RAFAEL|                        19|
|       136036|CENT EDUC DIST CA...|                        37|
|       136952|CENT EDUC DIST CA...|                        20|
|         4986|IE TECNICA PIO AL...|                        30|
|         5785|INSTITUCION EDUCA...|                        24|
|        41459|COLEGIO COMERCIAL...|                        11|
|       158725|           SIGLO XXI|                        13|
|       106120|EL NUEVO COL THE ...|                        47|
+-------------+--------------------+--------------------------+
only showing top 10 rows



In [67]:
print("El numero de colegios es: ", dfc.count())

El numero de colegios es:  365


## Colegios segundo semestre

In [68]:
# SE crea Dataframe de colegios primer semestre
dfc2 = spark.read.load(base + '/' + coleDic['20142'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='|')
dfc2 = dfc2.withColumn('COLE_COD_DANE', dfc2['COLE_COD_DANE'].cast("string"))

In [69]:
print(dfc2.printSchema())

root
 |-- PERIODO: integer (nullable = true)
 |-- COLE_COD_DANE: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: integer (nullable = true)
 |-- COLE_MPIO_MUNICIPIO: string (nullable = true)
 |-- COLE_COD_DEPTO: integer (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_GRADO: integer (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- MATRICULADOS_ULTIMOS_3: integer (nullable = true)
 |-- EVALUADOS_ULTIMOS_3: integer (nullable = true)
 |-- INDICE_MATEMATICAS: string (nullable = true)
 |-- INDICE_C_NATURALES: string (nullable = true)
 |-- INDICE_SOCIALES_CIUDADANAS: string (nullable = true)
 |-- INDICE_LECTURA_CRITICA: string (nullable = true)
 |-- INDICE_INGLES: string (nullable = true)
 |-- INDICE_TOTAL: string (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)

None


In [70]:
dfc2.select('COLE_COD_DANE','COLE_INST_NOMBRE','INDICE_TOTAL').show(10)

+-------------+--------------------+------------+
|COLE_COD_DANE|    COLE_INST_NOMBRE|INDICE_TOTAL|
+-------------+--------------------+------------+
| 205790000235| I. E. LA INMACULADA|      0,6112|
| 154128000019|INST EDUCATIVA CO...|      0,6924|
| 108433000051|INSTITUCION EDUCA...|      0,6012|
| 105147000568|I. E. JOSE MARIA ...|      0,6692|
| 105147000401|      I. E. COLOMBIA|      0,6408|
| 219110000636|INSTITUCION EDUCA...|      0,5284|
| 419110000970|          I.E. TIMBA|      0,6417|
| 111001104264|COLEGIO GERARDO M...|      0,6945|
| 205138000142|I. E. R. BUENOS A...|      0,6721|
| 276364001458|    I.E. SAN ANTONIO|       0,607|
+-------------+--------------------+------------+
only showing top 10 rows



In [71]:
print("El numero de colegios segundo semestre: ", dfc2.count())

El numero de colegios segundo semestre:  10927


## Anàlisis comparativo entre estudiantes y colegios primer semestre

In [72]:
dfjoin = df1.join(dfc, df1.COLE_COD_DANE_ESTABLECIMIENTO == dfc.COLE_COD_DANE)

In [73]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios: 0


In [74]:
dfjoinPorNom = df1.join(dfc, df1.COLE_NOMBRE_ESTABLECIMIENTO == dfc.COLE_INST_NOMBRE)

In [75]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios:", dfjoinPorNom.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios: 169


In [76]:
dfjoinPorNom.printSchema()

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: string (nullable = true)
 |-- COLE_NOMBRE_ESTABLECIMIENTO: string (nullable = false)
 |-- avg(PUNT_MATEMATICAS): double (nullable = true)
 |-- avg(PUNT_PROFUNDIZA_MATEMATICA): double (nullable = true)
 |-- avg(PUNT_CIENCIAS_SOCIALES): double (nullable = true)
 |-- avg(PUNT_PROFUNDIZA_CSOCIALES): double (nullable = true)
 |-- avg(PUNT_INGLES): double (nullable = true)
 |-- avg(PUNT_BIOLOGIA): double (nullable = true)
 |-- avg(PUNT_PROFUNDIZA_BIOLOGIA): double (nullable = true)
 |-- avg(PUNT_FILOSOFIA): double (nullable = true)
 |-- avg(PUNT_INTERDISC_MEDIOAMBIENTE): double (nullable = true)
 |-- avg(PUNT_FISICA): double (nullable = true)
 |-- avg(PUNT_QUIMICA): double (nullable = true)
 |-- avg(PUNT_LENGUAJE): double (nullable = true)
 |-- avg(PUNT_PROFUNDIZA_LENGUAJE): double (nullable = true)
 |-- avg(PUNT_INTERDISC_VIOLENCIAYSOC): double (nullable = true)
 |-- avg(Suma): double (nullable = true)
 |-- rank: integer (nullable = true)
 |-- PERIOD

In [107]:
# Se cruza la informaciòn por el nombre del colegio - se evidencia que el còdigo DANE no coincide
dfjoinPorNom.select('rank','COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO','COLE_COD_DANE').show(10)

+----+-----------------------------+---------------------------+-------------+
|rank|COLE_COD_DANE_ESTABLECIMIENTO|COLE_NOMBRE_ESTABLECIMIENTO|COLE_COD_DANE|
+----+-----------------------------+---------------------------+-------------+
|   2|                 368276000826|       COL LA QUINTA DEL...|        55129|
|   3|                 311769004292|       COL SAN MATEO APO...|        62497|
|   5|                 311848002424|       COL SAN CARLOS   ...|        19471|
|   6|                 311848000278|       GIMN VERMONT     ...|        25130|
|  10|                 366001003814|       COL FUNDACION LIC...|        77776|
|  11|                 376001016334|       COLEGIO INGLES DE...|        58719|
|  13|                 376001026968|            COLEGIO FREINET|       105742|
|  15|                 376001011464|       COLEGIO GIMNASIO ...|        55806|
|  18|                 376001002376|       COLEGIO HISPANOAM...|        17004|
|  20|                 305001015503|       COLEGIO L

## Anàlisis comparativo entre estudiantes y colegios segundo semestre

In [120]:
# SE crea Dataframe de colegios segundo semestre
dfc2 = spark.read.load(base + '/' + coleDic['20142'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='|')
dfc2 = dfc2.withColumn('COLE_COD_DANE', dfc2['COLE_COD_DANE'].cast("string"))

In [124]:
dfjoin2 = df2.join(dfc2, df2.COLE_COD_DANE_ESTABLECIMIENTO == dfc2.COLE_COD_DANE)

In [125]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes 2do semestre y colegios:", dfjoin2.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes 2do semestre y colegios: 10177


In [137]:
dfjoin2 = dfjoin2.withColumn("rankIndiceTotal", dense_rank().over(Window.orderBy(desc("INDICE_TOTAL"))))
dfjoin2.select('rank','rankIndiceTotal','avg(Suma)','COLE_NOMBRE_ESTABLECIMIENTO','COLE_COD_DANE','INDICE_TOTAL').show(10)

+----+-----------------+---------------------------+-------------+------------+
|rank|        avg(Suma)|COLE_NOMBRE_ESTABLECIMIENTO|COLE_COD_DANE|INDICE_TOTAL|
+----+-----------------+---------------------------+-------------+------------+
|   1|541.5833333333334|       COLOMBO BRITANICO...| 311001086320|      0,9101|
|   2|538.5714285714286|       CAMBRIDGE SCHOOL ...| 354518001139|      0,9104|
|   3|534.0888888888888|                LIC NAVARRA| 311001005176|      0,9011|
|   4|        530.40625|        INST ALBERTO MERANI| 311001089221|       0,913|
|   5|527.8615384615384|       COL NUEVO COLOMBO...| 311001076634|      0,9005|
|   6|526.3529411764706|            LIC CAMPO DAVID| 311001065489|      0,9063|
|   7|524.3674698795181|        COL ANGLO AMERICANO| 311848000308|      0,8967|
|   8|523.8055555555555|                 COL REFOUS| 425214000149|      0,8965|
|   9|519.9060402684564|        COLEGIO CORAZONISTA| 311001090921|      0,9001|
|  10|516.6190476190476|             COL